In this notebooks, we'll learn first how to combine multiple DataFrame using the `merge` method and then see how we can further explore the resulting DataFrame.

## Merging _pandas_ DataFrames

In [1]:
import pandas as pd

In [2]:
unemployment = pd.read_csv('../data/tn_unemployment.csv')

In [3]:
unemployment.head()

,laus_code,State,County,Name,Period,LF,Employed,Unemployed,unemployment_rate
0,CN4700100000000,47,1,"Anderson County, TN",Mar-21,34704,33010,1694,4.9
1,CN4700300000000,47,3,"Bedford County, TN",Mar-21,20623,19550,1073,5.2
2,CN4700500000000,47,5,"Benton County, TN",Mar-21,6723,6305,418,6.2
3,CN4700700000000,47,7,"Bledsoe County, TN",Mar-21,4252,3947,305,7.2
4,CN4700900000000,47,9,"Blount County, TN",Mar-21,64098,61119,2979,4.6


This unemployment data was obtained from the Burea of Labor Statistics.

Notice that it includes the labor force for each county in the LF column along with the number employed and number unemployed.

We can easily verify the unemployment rate values by dividing the Unemployed column by the LF column. Using `pandas`, we can do these types of operations on whole columns at a time:

In [4]:
unemployment['Unemployed'] / unemployment['LF']

0     0.048813
1     0.052029
2     0.062175
3     0.071731
4     0.046476
        ...   
90    0.053304
91    0.045889
92    0.049731
93    0.032985
94    0.039767
Length: 95, dtype: float64

We can also multiply an entire column by a number. For example, if we can to convert the proportions from above to percentages, we can do

In [5]:
100 * unemployment['Unemployed'] / unemployment['LF']

0     4.881282
1     5.202929
2     6.217462
3     7.173095
4     4.647571
        ...   
90    5.330424
91    4.588873
92    4.973107
93    3.298477
94    3.976700
Length: 95, dtype: float64

Now, let's bring in our second DataFrame.

In [6]:
population = pd.read_csv('../data/tn_population.csv')

In [7]:
population.head()

,County Name,Population
0,Anderson County,75129
1,Bedford County,45058
2,Benton County,16489
3,Bledsoe County,12876
4,Blount County,123010


Our goal is to combine the unemployment and population data. In order to do this, _pandas_ needs a common column to join on. 

Notice that the `Name` column from `unemployment` and the `County Name` column from `population` almost work, except that `Name` also includes ", TN". Let's remove this part so that we have a column to join on.

### Brief Detour - String Methods

When working with text data in `pandas`, it is often useful to utilize the built-in sting methods. To use these methods, you must prepend a `.str` before the desired method.

For example, we can make column entirely uppercase using the `upper()` method.

In [8]:
population['County Name'].str.upper()

0       ANDERSON COUNTY
1        BEDFORD COUNTY
2         BENTON COUNTY
3        BLEDSOE COUNTY
4         BLOUNT COUNTY
            ...        
90         WAYNE COUNTY
91       WEAKLEY COUNTY
92         WHITE COUNTY
93    WILLIAMSON COUNTY
94        WILSON COUNTY
Name: County Name, Length: 95, dtype: object

Another often useful method is the `replace()` method. To use this method, specify what pattern you want to replace and then the replacement text.

In [9]:
unemployment['Period'].str.replace('21', '2021')

0     Mar-2021
1     Mar-2021
2     Mar-2021
3     Mar-2021
4     Mar-2021
        ...   
90    Mar-2021
91    Mar-2021
92    Mar-2021
93    Mar-2021
94    Mar-2021
Name: Period, Length: 95, dtype: object

A method that we can use that will allow us to merge our dataframes is the `.str.split()` method.

Notice that if we split on the comma, the first piece will match what is contained in the `County Name` column.

In [10]:
unemployment['Name'].str.split(',')

0       [Anderson County,  TN]
1        [Bedford County,  TN]
2         [Benton County,  TN]
3        [Bledsoe County,  TN]
4         [Blount County,  TN]
                ...           
90         [Wayne County,  TN]
91       [Weakley County,  TN]
92         [White County,  TN]
93    [Williamson County,  TN]
94        [Wilson County,  TN]
Name: Name, Length: 95, dtype: object

By default, this method returns a list. We can make it return a DataFrame by using the `expand` argument.

In [11]:
unemployment['Name'].str.split(',', expand = True)

,0,1
0,Anderson County,TN
1,Bedford County,TN
2,Benton County,TN
3,Bledsoe County,TN
4,Blount County,TN
...,...,...
90,Wayne County,TN
91,Weakley County,TN
92,White County,TN
93,Williamson County,TN


We only want the first column.

In [12]:
unemployment['Name'].str.split(',', expand = True)[0]

0       Anderson County
1        Bedford County
2         Benton County
3        Bledsoe County
4         Blount County
            ...        
90         Wayne County
91       Weakley County
92         White County
93    Williamson County
94        Wilson County
Name: 0, Length: 95, dtype: object

Finally, we can assign this back to the `Name` column.

In [13]:
unemployment['Name'] = unemployment['Name'].str.split(',', expand = True)[0]

In [14]:
unemployment.head()

,laus_code,State,County,Name,Period,LF,Employed,Unemployed,unemployment_rate
0,CN4700100000000,47,1,Anderson County,Mar-21,34704,33010,1694,4.9
1,CN4700300000000,47,3,Bedford County,Mar-21,20623,19550,1073,5.2
2,CN4700500000000,47,5,Benton County,Mar-21,6723,6305,418,6.2
3,CN4700700000000,47,7,Bledsoe County,Mar-21,4252,3947,305,7.2
4,CN4700900000000,47,9,Blount County,Mar-21,64098,61119,2979,4.6


Now, we are ready to merge.

**Option 1:** Use the `left_on` and `right_on` arguments to tell pandas what to merge on.

In [15]:
pd.merge(left = population, right = unemployment, left_on = 'County Name', right_on = 'Name')

,County Name,Population,laus_code,State,County,Name,Period,LF,Employed,Unemployed,unemployment_rate
0,Anderson County,75129,CN4700100000000,47,1,Anderson County,Mar-21,34704,33010,1694,4.9
1,Bedford County,45058,CN4700300000000,47,3,Bedford County,Mar-21,20623,19550,1073,5.2
2,Benton County,16489,CN4700500000000,47,5,Benton County,Mar-21,6723,6305,418,6.2
3,Bledsoe County,12876,CN4700700000000,47,7,Bledsoe County,Mar-21,4252,3947,305,7.2
4,Blount County,123010,CN4700900000000,47,9,Blount County,Mar-21,64098,61119,2979,4.6
...,...,...,...,...,...,...,...,...,...,...,...
90,Wayne County,17021,CN4718100000000,47,181,Wayne County,Mar-21,6416,6074,342,5.3
91,Weakley County,35021,CN4718300000000,47,183,Weakley County,Mar-21,15494,14783,711,4.6
92,White County,25841,CN4718500000000,47,185,White County,Mar-21,12085,11484,601,5.0
93,Williamson County,202686,CN4718700000000,47,187,Williamson County,Mar-21,129484,125213,4271,3.3


Notice that we end up with a duplicated column (with different names).

**Option 2:** Rename the merging column for one of our DataFrames.

In [16]:
pd.merge(left = population,
         right = unemployment.rename(columns = {'Name': 'County Name'}))

,County Name,Population,laus_code,State,County,Period,LF,Employed,Unemployed,unemployment_rate
0,Anderson County,75129,CN4700100000000,47,1,Mar-21,34704,33010,1694,4.9
1,Bedford County,45058,CN4700300000000,47,3,Mar-21,20623,19550,1073,5.2
2,Benton County,16489,CN4700500000000,47,5,Mar-21,6723,6305,418,6.2
3,Bledsoe County,12876,CN4700700000000,47,7,Mar-21,4252,3947,305,7.2
4,Blount County,123010,CN4700900000000,47,9,Mar-21,64098,61119,2979,4.6
...,...,...,...,...,...,...,...,...,...,...
90,Wayne County,17021,CN4718100000000,47,181,Mar-21,6416,6074,342,5.3
91,Weakley County,35021,CN4718300000000,47,183,Mar-21,15494,14783,711,4.6
92,White County,25841,CN4718500000000,47,185,Mar-21,12085,11484,601,5.0
93,Williamson County,202686,CN4718700000000,47,187,Mar-21,129484,125213,4271,3.3


We can even select out just the columns that we need after the merge:

In [17]:
pd.merge(left = population,
         right = unemployment[['Name', 'unemployment_rate']].rename(columns = {'Name': 'County Name'}))

,County Name,Population,unemployment_rate
0,Anderson County,75129,4.9
1,Bedford County,45058,5.2
2,Benton County,16489,6.2
3,Bledsoe County,12876,7.2
4,Blount County,123010,4.6
...,...,...,...
90,Wayne County,17021,5.3
91,Weakley County,35021,4.6
92,White County,25841,5.0
93,Williamson County,202686,3.3


Once we're happy with the results, we can save them to a DataFrame.

In [18]:
tn_counties = pd.merge(left = population,
         right = unemployment[['Name', 'unemployment_rate']].rename(columns = {'Name': 'County Name'}))

Now, let's add in some additional information about these counties - which Grand Division they belong to.

In [19]:
grand_divisions = pd.read_csv('../data/tn_divisions.csv')

In [20]:
grand_divisions.head()

,County,Division
0,Anderson County,East
1,Bledsoe County,East
2,Blount County,East
3,Bradley County,East
4,Campbell County,East


**Your Turn:** Merge tn_counties with grand_divisions. Save the results back to `tn_counties`.

In [22]:
tn_counties = pd.merge(left = tn_counties, 
         right = grand_divisions.rename(columns = {'County': "County Name"}))

In [23]:
tn_counties.head()

,County Name,Population,unemployment_rate,Division
0,Anderson County,75129,4.9,East
1,Bedford County,45058,5.2,Middle
2,Benton County,16489,6.2,West
3,Bledsoe County,12876,7.2,East
4,Blount County,123010,4.6,East


## Further Methods of Slicing and Exploration

When working with numeric data, it is often useful to look at some summary statistics. The `describe()` method is an easy way to get a summary of a numeric column.

In [24]:
tn_counties['Population'].describe()

count        95.000000
mean      67006.400000
std      125888.117049
min        5077.000000
25%       18101.500000
50%       31807.000000
75%       56443.000000
max      927644.000000
Name: Population, dtype: float64

Let's now find all counties in East Tennessee that have at least 100,000 residents.

We saw last week how we could use `.loc` to filter a DataFrame. If there are multiple conditions we want to filter on, we can still use `.loc`, but we must first surround each condition in parentheses and separate the conditions by an &.

In [25]:
tn_counties.loc[(tn_counties['Division'] == 'East') & (tn_counties['Population'] >= 100000)]

,County Name,Population,unemployment_rate,Division
4,Blount County,123010,4.6,East
32,Hamilton County,336463,4.8,East
46,Knox County,432226,4.1,East
81,Sullivan County,156823,5.5,East
89,Washington County,122979,4.6,East


The & means that pandas will find all rows that meet both conditions. You can think of & as "and". If we want to find all rows that meet either condition a _or_ condition b, we can use the |, which can be thought of as "or".

For example, if we want to find any counties that have either at least 250,000 residents _or_ have unemployment less than 4%, we can accomplish this using the following code.

In [26]:
tn_counties.loc[(tn_counties['Population'] >= 250000) | (tn_counties['unemployment_rate'] < 4)]

,County Name,Population,unemployment_rate,Division
18,Davidson County,626681,4.6,Middle
32,Hamilton County,336463,4.8,East
46,Knox County,432226,4.1,East
63,Moore County,6362,3.7,Middle
74,Rutherford County,262604,4.1,Middle
78,Shelby County,927644,7.0,West
93,Williamson County,202686,3.3,Middle


If we want to filter down to the rows matching 2 or more conditions, we can either string together a series of conditions with | or we can use the `.isin` method and provide a list of values we want to match.

For example, if we want to look at the rows for Davidson, Shelby, Knox, and Hamilton Counties, we could accomplish it as follows:

In [27]:
metro_counties = ['Davidson County', 'Shelby County', 'Knox County', 'Hamilton County']

tn_counties.loc[tn_counties['County Name'].isin(metro_counties)]

,County Name,Population,unemployment_rate,Division
18,Davidson County,626681,4.6,Middle
32,Hamilton County,336463,4.8,East
46,Knox County,432226,4.1,East
78,Shelby County,927644,7.0,West


Finally, if we want to negate a condition, we can do so using a ~.

For example, let's say we want to find all counties besides the four metro counties above.

In [28]:
tn_counties.loc[~tn_counties['County Name'].isin(metro_counties)]

,County Name,Population,unemployment_rate,Division
0,Anderson County,75129,4.9,East
1,Bedford County,45058,5.2,Middle
2,Benton County,16489,6.2,West
3,Bledsoe County,12876,7.2,East
4,Blount County,123010,4.6,East
...,...,...,...,...
90,Wayne County,17021,5.3,Middle
91,Weakley County,35021,4.6,West
92,White County,25841,5.0,Middle
93,Williamson County,202686,3.3,Middle
